In [0]:
!git clone https://github.com/pruvi007/Age-Estimation-Models

Cloning into 'Age-Estimation-Models'...
remote: Enumerating objects: 2225, done.
remote: Total 2225 (delta 0), reused 0 (delta 0), pack-reused 2225
Receiving objects: 100% (2225/2225), 88.94 MiB | 54.57 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [0]:
import os
import cv2
import numpy as np

In [0]:
def hotVector(n,classes):
    vector = [ 0 for i in range(classes) ]
    vector[n-1] = 1
    vector = np.array(vector)
    return vector

In [0]:
# get the images from directory
dir = "Age-Estimation-Models/FGNET/FGNET/images"
images = []
for file in os.listdir(dir):
    img = cv2.imread(dir+"/"+file,0)
    images.append(img)
images = np.array(images)
print(np.shape(images[0]))
# resize the images for uniformity
min_x, min_y = 1000,1000
for img in images:
    min_x, min_y = min(min_x,np.shape(img)[0]), min(min_y,np.shape(img)[1])
print(min_x,min_y)
for i in range(len(images)):
    width = min_x
    height = min_y
    dim = (width, height)
    resized = cv2.resize(images[i], dim, interpolation = cv2.INTER_AREA)
    images[i] = resized

ages = []
for file in os.listdir(dir):
    s = file.split('.')
    a = []
    n = len(s[0])
    for i in range(n-1,-1,-1):
        if s[0][i]=='A':
            break
        a.append(s[0][i])
    if a[0] in ['a','b','c','d','e']:
        a.pop(0)
    age = 0
    for i in range(len(a)-1,-1,-1):
        age = age*10 + int(a[i])
    
    # print(file)
    # print(age)
    ages.append(hotVector(age,100))
ages = np.array(ages)
print(np.shape(ages))

(448, 409)
359 300
(1002, 100)


In [0]:
from sklearn.neural_network import MLPClassifier
perc = 50
train_x = []
train_y = []
for i in range((len(images)*perc)//100):
    train_x.append( images[i].flatten() )
    train_y.append( ages[i] )

# print(np.shape(train_x),np.shape(train_y))
mlp = MLPClassifier(hidden_layer_sizes=(1024,512,100), activation='relu', solver='adam', alpha=0.0001, batch_size='auto' )
mlp.fit(train_x,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(1024, 512, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
mae = 0
for i in range((len(images)*perc)//100,len(images)):
    f = images[i].flatten().reshape(1, -1)
    p_age = mlp.predict(f)
    # print("Predicted:{}, Actual:{}".format(np.argmax(p_age),np.argmax(ages[i])))
    mae += abs( np.argmax(p_age)-np.argmax(ages[i]) )
print("MAE:",mae/len(images))

MAE: 9.258483033932135
